DATASET

In [72]:
import shutil
import pandas as pd
import numpy as np
import cv2
import os

In [73]:
lfw_allnames = pd.read_csv('lfw_allnames.csv')
lfw_allnames = lfw_allnames[lfw_allnames['images'] >= 20]
print(len(lfw_allnames))

lfw_allnames = lfw_allnames.sample(n=5)
print(lfw_allnames)

lfw_allnames.to_csv('lfw_224.csv', index=False)
names = lfw_allnames['name'].tolist()

os.mkdir('lfw_224')

for name in names:
    shutil.copytree('lfw-deepfunneled/' + name, 'lfw_224/' + name)

62
                      name  images
210           Alvaro_Uribe      35
4551  Recep_Tayyip_Erdogan      30
5354           Tiger_Woods      23
5659          Winona_Ryder      24
248           Andre_Agassi      36


In [74]:
for name in names:
    images = os.listdir('lfw_224/' + name)
    for image in images:
        img = cv2.imread('lfw_224/' + name + '/' + image)
        img = cv2.resize(img, (224, 224))
        cv2.imwrite('lfw_224/' + name + '/' + image, img)

In [75]:
os.mkdir('lfw_224/train')
os.mkdir('lfw_224/test')

for name in names:
    images = os.listdir('lfw_224/' + name)
    images = np.random.choice(images, 10, replace=False)
    os.mkdir('lfw_224/test/' + name)
    for image in images:
        shutil.move('lfw_224/' + name + '/' + image, 'lfw_224/test/' + name + '/' + image)

for name in names:
    os.mkdir('lfw_224/train/' + name)
    images = os.listdir('lfw_224/' + name)
    for image in images:
        shutil.move('lfw_224/' + name + '/' + image, 'lfw_224/train/' + name + '/' + image)
        
for name in names:
    os.rmdir('lfw_224/' + name)

In [76]:
images = os.listdir('me')
for image in images:
    img = cv2.imread('me/' + image)
    img = cv2.resize(img, (224, 224))
    cv2.imwrite('me/' + image, img)
    
os.mkdir('lfw_224/train/Luke')
os.mkdir('lfw_224/test/Luke')
images = os.listdir('me')
images = np.random.choice(images, 6, replace=False)
for image in images:
    shutil.copy('me/' + image, 'lfw_224/test/Luke/' + image)
images = os.listdir('me')
for image in images:
    shutil.copy('me/' + image, 'lfw_224/train/Luke/' + image)

MODEL

In [85]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os
from keras.applications import ResNet50

In [86]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    channel_shift_range=25,  
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'lfw_224/train',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)

val_generator = test_datagen.flow_from_directory(
    'lfw_224/test',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 112 images belonging to 6 classes.
Found 56 images belonging to 6 classes.


In [92]:
input_shape=(224,224,3)
num_of_identities = len(os.listdir('lfw_224/train'))

# model to predict 10 identities
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_of_identities, activation='softmax')
])

In [93]:
optimizer = Adam(learning_rate=0.001)
loss = CategoricalCrossentropy()

# accuracy, top-k categorical accuracy, precision, recall, AUC
metrics = ['accuracy', 'top_k_categorical_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [94]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 26, 26, 128)     

In [95]:
file_path="face_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='accuracy', save_best_only=True)

In [ ]:
model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[checkpoint]
    )

In [100]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model('face_model.h5')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'lfw_224/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
    )

model.evaluate(test_generator)
model.evaluate(train_generator)

Found 56 images belonging to 6 classes.
7/7 [==============================] - 1s 146ms/step - loss: 0.2346 - accuracy: 0.9196 - top_k_categorical_accuracy: 1.0000 - precision_3: 0.9273 - recall_3: 0.9107 - auc_3: 0.9937


[0.23463329672813416,
 0.9196428656578064,
 1.0,
 0.9272727370262146,
 0.9107142686843872,
 0.9936782121658325]